In [2]:
import os
import numpy as np
import nibabel as nib
from nibabel.processing import resample_from_to
from nibabel.orientations import aff2axcodes
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

def ensure_folder_exists(folder_path):
    """Create folder if it doesn't exist"""
    if not os.path.exists(folder_path):
        os.makedirs(folder_path, exist_ok=True)
    return folder_path

def load_and_process_nifti(file_path):
    """Load and preprocess NIFTI file with proper orientation"""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    img = nib.load(file_path)
    img = nib.as_closest_canonical(img)
    print(f"Image orientation: {aff2axcodes(img.affine)}")
    
    data = img.get_fdata()
    data = np.transpose(data, (1, 0, 2))
    data = np.rot90(data, k=2, axes=(0, 1))
    
    return img, data

def align_mask_to_base(base_img, mask_img):
    """Align mask to base image using nibabel's resampling"""
    base_ras = nib.as_closest_canonical(base_img)
    mask_ras = nib.as_closest_canonical(mask_img)
    mask_resampled = resample_from_to(mask_ras, base_ras)
    
    mask_data = mask_resampled.get_fdata()
    mask_data = np.transpose(mask_data, (1, 0, 2))
    mask_data = np.rot90(mask_data, k=2, axes=(0, 1))
    binary_mask = (mask_data > 0.5).astype(np.uint8)
    
    return binary_mask

def visualize_slice(image_data, masks, output_path, subject_id, phase_type):
    """Create a visualization of a middle slice with mask overlays"""
    middle_slice_idx = image_data.shape[2] // 2
    slice_data = image_data[:, :, middle_slice_idx]
    
    plt.figure(figsize=(12, 8))
    plt.imshow(slice_data, cmap='gray')
    
    colors = ['red', 'blue', 'green', 'yellow']
    
    for (name, mask_data), color in zip(masks.items(), colors):
        mask_slice = mask_data[:, :, middle_slice_idx]
        plt.imshow(mask_slice, alpha=0.3, cmap=LinearSegmentedColormap.from_list('custom', ['none', color]))
    
    plt.title(f"{subject_id} - {phase_type} (Slice {middle_slice_idx})")
    plt.savefig(output_path)
    plt.close()
    print(f"Saved visualization to: {output_path}")

def create_phase_image(water_path, fat_path, output_dir, sbt_number, mask_paths, is_in_phase):
    """Create In Phase or Out Phase image from Water and Fat"""
    try:
        phase_type = "InPhase" if is_in_phase else "OutPhase"
        print(f"\nProcessing SBT{sbt_number:03d} with {phase_type}")
        
        # Load Water and Fat images
        water_img, water_data = load_and_process_nifti(water_path)
        fat_img, fat_data = load_and_process_nifti(fat_path)
        
        print(f"Water data shape: {water_data.shape}")
        print(f"Fat data shape: {fat_data.shape}")
        
        # Calculate phase data
        if is_in_phase:
            phase_data = water_data + fat_data
        else:
            phase_data = water_data - fat_data
        
        # Normalize to 16-bit range
        phase_data = ((phase_data - phase_data.min()) / 
                     (phase_data.max() - phase_data.min()) * 65535).astype(np.uint16)
        
        # Create output directory (no phase type in folder name)
        subject_id = f"SBT{sbt_number:03d}"
        subject_dir = ensure_folder_exists(os.path.join(output_dir, subject_id))
        
        # Save phase image
        phase_path = os.path.join(subject_dir, f"{subject_id}_images.nii")
        phase_img = nib.Nifti1Image(phase_data, water_img.affine)
        nib.save(phase_img, phase_path)
        print(f"Saved {phase_type} image: {phase_path}")
        
        # Process masks
        processed_masks = {}
        for name, mask_path in mask_paths.items():
            print(f"\nProcessing mask: {name}")
            mask_img = nib.load(mask_path)
            aligned_mask = align_mask_to_base(water_img, mask_img)
            
            # Save mask
            mask_output_name = f"{subject_id}_{name}_II.nii"
            mask_output_path = os.path.join(subject_dir, mask_output_name)
            mask_img_out = nib.Nifti1Image(aligned_mask, water_img.affine)
            nib.save(mask_img_out, mask_output_path)
            print(f"Saved mask to: {mask_output_path}")
            
            processed_masks[name] = aligned_mask
        
        # Save visualization
        vis_output_path = os.path.join(subject_dir, f"{subject_id}_visualization.png")
        visualize_slice(phase_data, processed_masks, vis_output_path, subject_id, phase_type)
        
        return True
        
    except Exception as e:
        print(f"Error processing SBT{sbt_number:03d}: {str(e)}")
        return False

def process_subject(subject_id, base_dir, start_sbt_number):
    """Process a single subject, creating two SBT folders"""
    try:
        # Construct file paths
        subject_dir = os.path.join(base_dir, subject_id)
        water_path = os.path.join(subject_dir, f"{subject_id}_IDEALWater.nii")
        fat_path = os.path.join(subject_dir, f"{subject_id}_IDEALFat.nii")
        
        # Define mask paths
        mask_paths = {
            "L_ES": os.path.join(subject_dir, f"{subject_id}_L_ES_MASK.nii"),
            "L_Mult": os.path.join(subject_dir, f"{subject_id}_L_Multifidus_MASK.nii"),
            "R_ES": os.path.join(subject_dir, f"{subject_id}_R_ES_MASK.nii"),
            "R_Mult": os.path.join(subject_dir, f"{subject_id}_R_Multifidus_MASK.nii")
        }
        
        # Verify all files exist
        all_paths = [water_path, fat_path] + list(mask_paths.values())
        for path in all_paths:
            if not os.path.exists(path):
                print(f"Warning: File not found: {path}")
                return False
        
        processed_dir = "/Volumes/advent/processed_data"
        
        # First SBT number gets Out Phase
        success_out = create_phase_image(water_path, fat_path, processed_dir, 
                                       start_sbt_number, mask_paths, False)
        
        # Second SBT number gets In Phase
        success_in = create_phase_image(water_path, fat_path, processed_dir, 
                                      start_sbt_number + 1, mask_paths, True)
        
        return success_out and success_in
        
    except Exception as e:
        print(f"Error processing subject {subject_id}: {str(e)}")
        return False

def test_single_subject(subject_id, base_dir, start_sbt_number):
    """Test the processing pipeline on a single subject"""
    print(f"\n=== Testing processing for {subject_id} ===")
    print(f"Will create SBT{start_sbt_number:03d} (Out Phase) and SBT{start_sbt_number+1:03d} (In Phase)")
    
    try:
        success = process_subject(subject_id, base_dir, start_sbt_number)
        if success:
            print("\nProcessing successful! Created folders:")
            print(f"SBT{start_sbt_number:03d} - Out Phase")
            print(f"SBT{start_sbt_number+1:03d} - In Phase")
        else:
            print("\nProcessing failed. Check the error messages above.")
    except Exception as e:
        print(f"\nError during testing: {str(e)}")

def main():
    """Process all subjects"""
    base_dir = "/Volumes/advent/research/MRI/Marines/Analysis2"
    
    # Get all subject directories
    subject_dirs = sorted([d for d in os.listdir(base_dir) 
                         if os.path.isdir(os.path.join(base_dir, d))
                         and d.startswith("LS")])
    
    current_sbt = 94  # Start with SBT094
    for subject_id in subject_dirs:
        print(f"\nProcessing {subject_id} -> SBT{current_sbt:03d}/SBT{current_sbt+1:03d}")
        success = process_subject(subject_id, base_dir, current_sbt)
        if success:
            print(f"Successfully processed {subject_id}")
            current_sbt += 2  # Increment by 2 for next subject
        else:
            print(f"Failed to process {subject_id}")

if __name__ == "__main__":
    # Test on a single subject
    # Uncomment the next line to test a single subject:
    # test_single_subject("LS011", "/Volumes/advent/research/MRI/Marines/Analysis2", 94)
    
    # Comment out the next line when testing single subject:
    main()


Processing LS002 -> SBT094/SBT095
Failed to process LS002

Processing LS003 -> SBT094/SBT095
Failed to process LS003

Processing LS004 -> SBT094/SBT095
Failed to process LS004

Processing LS006 -> SBT094/SBT095
Failed to process LS006

Processing LS007 -> SBT094/SBT095

Processing SBT094 with OutPhase
Image orientation: ('R', 'A', 'S')
Image orientation: ('R', 'A', 'S')
Water data shape: (256, 176, 256)
Fat data shape: (256, 176, 256)
Saved OutPhase image: /Volumes/advent/processed_data/SBT094/SBT094_images.nii

Processing mask: L_ES
Saved mask to: /Volumes/advent/processed_data/SBT094/SBT094_L_ES_II.nii

Processing mask: L_Mult
Saved mask to: /Volumes/advent/processed_data/SBT094/SBT094_L_Mult_II.nii

Processing mask: R_ES
Saved mask to: /Volumes/advent/processed_data/SBT094/SBT094_R_ES_II.nii

Processing mask: R_Mult
Saved mask to: /Volumes/advent/processed_data/SBT094/SBT094_R_Mult_II.nii
Saved visualization to: /Volumes/advent/processed_data/SBT094/SBT094_visualization.png

Proces